In [1]:
import pandas as pd
dataset = pd.read_csv('Housing.csv')

In [2]:
dataset

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,502,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,503,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,504,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,505,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [3]:
print(dataset.shape)

(506, 15)


In [4]:
dataset[:5]

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [32]:
dataset = pd.concat([dataset['medv'],dataset.drop(['medv'], axis=1)],axis=1)
dataset[:1]

,medv,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,24.0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98


In [6]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

In [7]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv',index=False, header=False)

In [8]:
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket()

In [9]:
prefix = 'boston-housing'
training_data_path = sess.upload_data(
 path='training_dataset.csv',
 key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(
 path='validation_dataset.csv',
 key_prefix=prefix + '/input/validation')
print(training_data_path)
print(validation_data_path)

s3://sagemaker-us-west-2-373913539787/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-west-2-373913539787/boston-housing/input/validation/validation_dataset.csv


In [10]:
import boto3
from sagemaker import image_uris

In [11]:
region = boto3.Session().region_name
container = image_uris.retrieve('xgboost', region,
 version='latest')

In [15]:
from sagemaker.estimator import Estimator
xgb_estimator = Estimator(
 container,
 role=sagemaker.get_execution_role(),
 instance_count=1,
 instance_type='ml.m4.xlarge',
 output_path='s3://{}/{}/output'.format(bucket,
 prefix))

In [16]:
xgb_estimator.set_hyperparameters(
 objective='reg:linear',
 num_round=200,
 early_stopping_rounds=10)

In [17]:
training_data_channel = sagemaker.TrainingInput(
 s3_data=training_data_path,
 content_type='text/csv')

In [18]:
validation_data_channel = sagemaker.TrainingInput(
 s3_data=validation_data_path,
 content_type='text/csv')

In [20]:
xgb_estimator.fit({'train': training_data_channel,
 'validation': validation_data_channel})

2021-04-08 17:45:36 Starting - Starting the training job...
2021-04-08 17:46:00 Starting - Launching requested ML instancesProfilerReport-1617903936: InProgress
......
2021-04-08 17:47:05 Starting - Preparing the instances for training.........
2021-04-08 17:48:25 Downloading - Downloading input data
2021-04-08 17:48:25 Training - Downloading the training image..Arguments: train
[2021-04-08:17:48:46:INFO] Running standalone xgboost training.
[2021-04-08:17:48:46:INFO] File size need to be processed in the node: 0.04mb. Available memory size in the node: 8431.5mb
[2021-04-08:17:48:46:INFO] Determined delimiter of CSV input is ','
[17:48:46] S3DistributionType set as FullyReplicated
[17:48:46] 455x14 matrix with 6370 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-04-08:17:48:46:INFO] Determined delimiter of CSV input is ','
[17:48:46] S3DistributionType set as FullyReplicated
[17:48:46] 51x14 matrix with 714 entries loaded from /opt/ml/input/data

In [21]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())
endpoint_name = 'xgb-demo'+'-'+timestamp
xgb_predictor = xgb_estimator.deploy(
 endpoint_name=endpoint_name,
 initial_instance_count=1,
 instance_type='ml.t2.medium')

---------------!

In [45]:
# test_sample = '0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98'
test_samples = [
'0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.9,4.98',
'0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.9,9.14']

In [46]:
xgb_predictor.CONTENT_TYPE = 'text/csv'
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgb_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

In [47]:
response = xgb_predictor.predict(test_sample)

In [49]:
response

[['43.45135498046875']]

In [44]:
dataset[:2]

,medv,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,24.0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.9,4.98
1,21.6,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14
